In [70]:
import pandas as pd
import os
import nltk
import tqdm
import numpy as np

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer

from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords  

from tensorflow import keras
from tensorflow.keras import Input, Model, Sequential
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import plot_model, to_categorical
from sklearn.metrics import log_loss,accuracy_score

from sklearn.model_selection import StratifiedKFold

In [14]:
trn=pd.read_csv('data/train.csv',index_col=0)
tst=pd.read_csv('data/test_x.csv',index_col=0)

In [66]:
trn['author'].value_counts()

3    15063
0    13235
2    11554
4     7805
1     7222
Name: author, dtype: int64

In [22]:
trn.iloc[0].text

'He was almost choking. There was so much, so much he wanted to say, but strange exclamations were all that came from his lips. The Pole gazed fixedly at him, at the bundle of notes in his hand; looked at odin, and was in evident perplexity.'

In [71]:
n_class=5
n_fold=5
seed=32152339
cv = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=seed)

# tokenization

In [33]:
trn["token"]=trn["text"].apply(word_tokenize)

# lemmatizaion

In [36]:
lemmatizer=WordNetLemmatizer()

In [54]:
for i in range(len(trn['token'])):
    trn['token'][i]=[lemmatizer.lemmatize(t) for t in trn['token'][i]]

C:\Users\ATIV\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


# TfidfVectorizer로 token화, 불용어제거, 실수형변환

In [59]:
vec=TfidfVectorizer(tokenizer=word_tokenize, stop_words=stopwords.words('english'),min_df=50)
X_cnt=vec.fit_transform(trn['text'])
print(X_cnt.shape)

C:\Users\ATIV\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:386: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'d", "'ll", "'re", "'s", "'ve", 'could', 'might', 'must', "n't", 'need', 'sha', 'wo', 'would'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


(54879, 2911)


In [68]:
X_cnt_tst=vec.fit_transform(tst['text'])
print(X_cnt_tst.shape)

(19617, 2379)


In [61]:
X_cnt.todense()

matrix([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.18298108,
         0.18845654],
        [0.        , 0.        , 0.        , ..., 0.08014345, 0.        ,
         0.        ],
        ...,
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.20204159,
         0.20808741],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ]])

# 신경망 학습

In [72]:
def get_model():
    model = Sequential()

    model.add(Dense(units=64, activation='relu'))
    model.add(Dense(units=n_class, activation='softmax'))
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam')
    return model

In [78]:
X_trn,X_val,y_trn,y_val = train_test_split(X_cnt,trn['author'],test_size=0.3, random_state=seed)

In [83]:
clf = get_model()

es = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=5,
                   verbose=1, mode='min', baseline=None, restore_best_weights=True)

rlr = ReduceLROnPlateau(monitor='val_loss', factor=0.5,
                        patience=3, min_lr=1e-6, mode='min', verbose=1)

clf.fit(X_trn.todense(), 
        to_categorical(y_trn),
        validation_data=(X_val.todense(), to_categorical(y_val)),
        epochs=2, # 에포크는 일반적으로 10 정도로 둠
        callbacks=[es, rlr])

Train on 38415 samples, validate on 16464 samples
Epoch 1/2
38415/38415 [==============================] - ETA: 13:24:18 - loss: 1.611 - ETA: 6:42:19 - loss: 1.609 - ETA: 2:13:50 - loss: 1.61 - ETA: 1:12:47 - loss: 1.60 - ETA: 49:54 - loss: 1.6036 - ETA: 34:33 - loss: 1.60 - ETA: 27:18 - loss: 1.59 - ETA: 21:16 - loss: 1.59 - ETA: 17:24 - loss: 1.58 - ETA: 14:41 - loss: 1.58 - ETA: 12:41 - loss: 1.57 - ETA: 11:09 - loss: 1.56 - ETA: 9:56 - loss: 1.5625 - ETA: 9:04 - loss: 1.557 - ETA: 8:13 - loss: 1.553 - ETA: 7:36 - loss: 1.547 - ETA: 7:04 - loss: 1.541 - ETA: 6:32 - loss: 1.535 - ETA: 6:07 - loss: 1.530 - ETA: 5:42 - loss: 1.523 - ETA: 5:25 - loss: 1.519 - ETA: 5:08 - loss: 1.512 - ETA: 4:52 - loss: 1.506 - ETA: 4:35 - loss: 1.499 - ETA: 4:18 - loss: 1.492 - ETA: 4:03 - loss: 1.483 - ETA: 3:51 - loss: 1.477 - ETA: 3:38 - loss: 1.468 - ETA: 3:26 - loss: 1.458 - ETA: 3:15 - loss: 1.449 - ETA: 3:09 - loss: 1.442 - ETA: 3:03 - loss: 1.436 - ETA: 2:57 - loss: 1.430 - ETA: 2:51 - loss: 1.4

In [84]:
p_val= clf.predict(X_val.todense())

In [101]:
len(np.argmax(p_val,axis=1))

16464

In [121]:
log_loss(y_val,p_val)

0.7205332337415213

In [119]:
accuracy_score(pd.Series(np.array(y_val)),pd.Series(np.argmax(p_val,axis=1)))

0.7298955296404276